In [ ]:
#ASIL
import torch
import torch.nn as nn

class Swish(nn.Module):
    def forward(self, x):
        return x * torch.sigmoid(x)

#CNN MODEL
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)

        self.conv4 = nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1)
        self.conv5 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1)
        self.conv6 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1)

        self.conv7 = nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1)
        self.conv8 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)
        self.conv9 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)
        self.swish = Swish()
        self.avg_pool = nn.AvgPool2d(4)
        self.fc1 = nn.Linear(1024, 1024)
        self.fc2 = nn.Linear(1024, num_classes)

    def forward(self, x):
        c1 = self.swish(self.conv1(x))
        c2 = self.swish(self.conv2(c1))
        c3 = self.swish(self.conv3(c2))
        out1 = c1 + c3

        c4 = self.swish(self.conv4(out1))
        c5 = self.swish(self.conv5(c4))
        c6 = self.swish(self.conv6(c5))
        out2 = c4 + c6

        c7 = self.swish(self.conv7(out2))
        c8 = self.swish(self.conv8(c7))
        c9 = self.swish(self.conv9(c8))
        out3 = c7 + c9

        out4 = self.avg_pool(out3)
        out5 = out4.view(out4.size(0), -1)
        out6 = self.fc1(out5)
        out7 = self.fc2(out6)

        return out7


# Load the dataset
transform = transforms.Compose([transforms.Resize((32, 32)), transforms.ToTensor()])
train_set = datasets.ImageFolder(root='/content/drive/MyDrive/CaltechTinySplit/train', transform=transform)
test_set = datasets.ImageFolder(root='/content/drive/MyDrive/CaltechTinySplit/test', transform=transform)
val_set = datasets.ImageFolder(root='/content/drive/MyDrive/CaltechTinySplit/val', transform=transform)

train_loader = DataLoader(train_set, batch_size=16, shuffle=True)
test_loader = DataLoader(test_set, batch_size=16, shuffle=False)
val_loader = DataLoader(val_set, batch_size=16, shuffle=False)

# Initialize the model, optimizer, and loss function
model = CNN(num_classes=9)
optimizer = Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

# Train the model
for epoch in range(60):
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

# Save the model
torch.save(model.state_dict(), 'model.pth')

# Load and evaluate the model
model.load_state_dict(torch.load('model.pth'))
model.eval()

# Evaluate the model using the test dataset
correct = 0
total = 0
all_labels = []
all_predicted = []

with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        all_labels.extend(labels.numpy())
        all_predicted.extend(predicted.numpy())

accuracy = 100 * correct / total
print(f'Accuracy: {accuracy:.2f}%')

# Calculate confusion matrix and F-score
conf_matrix = confusion_matrix(all_labels, all_predicted)
f_score = f1_score(all_labels, all_predicted, average='macro')

print(f'Confusion Matrix:\n{conf_matrix}')
print(f'F-score: {f_score:.4f}')

In [ ]:
#Her epoch için loss fonksiyonu, accuracy ve f-score görüntülemek için
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
from sklearn.metrics import confusion_matrix, f1_score
import numpy as np

class Swish(nn.Module):
    def forward(self, x):
        return x * torch.sigmoid(x)

# CNN
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)

        self.conv4 = nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1)
        self.conv5 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1)
        self.conv6 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1)

        self.conv7 = nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1)
        self.conv8 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)
        self.conv9 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)
        self.swish = Swish()
        self.avg_pool = nn.AvgPool2d(4)
        self.fc1 = nn.Linear(1024, 1024)
        self.fc2 = nn.Linear(1024, num_classes)

    def forward(self, x):
        c1 = self.swish(self.conv1(x))
        c2 = self.swish(self.conv2(c1))
        c3 = self.swish(self.conv3(c2))
        out1 = c1 + c3

        c4 = self.swish(self.conv4(out1))
        c5 = self.swish(self.conv5(c4))
        c6 = self.swish(self.conv6(c5))
        out2 = c4 + c6

        c7 = self.swish(self.conv7(out2))
        c8 = self.swish(self.conv8(c7))
        c9 = self.swish(self.conv9(c8))
        out3 = c7 + c9

        out4 = self.avg_pool(out3)
        out5 = out4.view(out4.size(0), -1)
        out6 = self.fc1(out5)
        out7 = self.fc2(out6)

        return out7

# Load the dataset
transform = transforms.Compose([transforms.Resize((32, 32)), transforms.ToTensor()])
train_set = datasets.ImageFolder(root='/content/drive/MyDrive/CaltechTinySplit/train', transform=transform)
test_set = datasets.ImageFolder(root='/content/drive/MyDrive/CaltechTinySplit/test', transform=transform)
val_set = datasets.ImageFolder(root='/content/drive/MyDrive/CaltechTinySplit/val', transform=transform)

train_loader = DataLoader(train_set, batch_size=16, shuffle=True)
test_loader = DataLoader(test_set, batch_size=16, shuffle=False)
val_loader = DataLoader(val_set, batch_size=16, shuffle=False)

# Initialize the model, optimizer, and loss function
model = CNN(num_classes=9)
optimizer = Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

# Train the model
for epoch in range(60):
    total_loss = 0  # to keep track of the total loss for each epoch
    correct = 0
    total = 0
    all_labels = []
    all_predicted = []

    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Calculate training accuracy and store predictions for later use
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        all_labels.extend(labels.numpy())
        all_predicted.extend(predicted.numpy())

        # Accumulate the total loss for the epoch
        total_loss += loss.item()

    # Calculate average loss for the epoch
    average_loss = total_loss / len(train_loader)

    # Calculate accuracy for the epoch
    accuracy = 100 * correct / total

    # Calculate F-score for the epoch
    f_score = f1_score(all_labels, all_predicted, average='macro')

    # Print metrics for the epoch
    print(f'Epoch [{epoch + 1}/60], Loss: {average_loss:.4f}, Accuracy: {accuracy:.2f}%, F-score: {f_score:.4f}')

# Save the model
torch.save(model.state_dict(), 'model.pth')

# Load and evaluate the model
model.load_state_dict(torch.load('model.pth'))
model.eval()

# Evaluate the model using the test dataset
correct = 0
total = 0
all_labels = []
all_predicted = []

with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        all_labels.extend(labels.numpy())
        all_predicted.extend(predicted.numpy())

# Calculate accuracy, confusion matrix, and F-score for the test dataset
accuracy = 100 * correct / total
conf_matrix = confusion_matrix(all_labels, all_predicted)
f_score = f1_score(all_labels, all_predicted, average='macro')

print(f'Test Accuracy: {accuracy:.2f}%')
print(f'Confusion Matrix:\n{conf_matrix}')
print(f'F-score: {f_score:.4f}')
